In [1]:
import requests
import json
import pandas as pd

In [2]:
import numpy as np

In [3]:
from PIL import Image
from io import BytesIO

In [4]:
pd.options.display.max_columns = 999

In [5]:
fpl = 'https://fantasy.premierleague.com/api'
static_end = '/bootstrap-static/'
fixtures_end = '/fixtures/'
player_end = '/element-summary/'

### static information

In [6]:
r = requests.get(fpl+static_end)
static = json.loads(r.content)

In [7]:
players = pd.DataFrame(static['elements'])

In [8]:
events = pd.DataFrame(static['events'])

In [9]:
teams = pd.DataFrame(static['teams'])
teams['id'] = teams['id'].astype(str)

### fixtures

In [10]:
r = requests.get(fpl+fixtures_end)
fixtures = json.loads(r.content)

In [11]:
fixtures_df = pd.DataFrame(fixtures)
fixtures_df['event'].fillna(0,inplace=True)
fixtures_df['event'] = fixtures_df['event'].astype(str)
fixtures_df['team_a'] = fixtures_df['team_a'].astype(str)
fixtures_df['team_h'] = fixtures_df['team_h'].astype(str)

### player summary

In [12]:
player_id = '355/'
r = requests.get(fpl+player_end+player_id)
player_summary = json.loads(r.content)

In [13]:
player_summary.keys()

dict_keys(['fixtures', 'history', 'history_past'])

In [30]:
player_summary['history'][0]

{'element': 355,
 'fixture': 1,
 'opponent_team': 6,
 'total_points': 13,
 'was_home': False,
 'kickoff_time': '2023-08-11T19:00:00Z',
 'team_h_score': 0,
 'team_a_score': 3,
 'round': 1,
 'minutes': 79,
 'goals_scored': 2,
 'assists': 0,
 'clean_sheets': 1,
 'goals_conceded': 0,
 'own_goals': 0,
 'penalties_saved': 0,
 'penalties_missed': 0,
 'yellow_cards': 0,
 'red_cards': 0,
 'saves': 0,
 'bonus': 3,
 'bps': 56,
 'influence': '67.6',
 'creativity': '23.7',
 'threat': '48.0',
 'ict_index': '13.9',
 'starts': 1,
 'expected_goals': '0.60',
 'expected_assists': '0.05',
 'expected_goal_involvements': '0.65',
 'expected_goals_conceded': '0.23',
 'value': 140,
 'transfers_balance': 0,
 'selected': 7200159,
 'transfers_in': 0,
 'transfers_out': 0}

### gameweek live

In [14]:
live_end = '/event/1/live/'
r = requests.get(fpl+live_end)
gw = json.loads(r.content)

In [15]:
id_list = []
stat_list = []
for player in gw['elements']:
    id_list.append(player['id'])
    stat_list.append(player['stats'])
stats_df = pd.DataFrame(stat_list)
stats_df.insert(0, 'id', id_list)

### sandbox

In [16]:
teams_dict= teams.set_index('id')['short_name'].to_dict()

In [17]:
def get_teams_difficulty(gw=None):
    if not gw:
        gw = events.loc[events['is_current'],['id']].values[0][0]
    team_h = fixtures_df.loc[fixtures_df['event'] == gw, ['team_h', 'team_h_difficulty']].values
    team_a = fixtures_df.loc[fixtures_df['event'] == gw, ['team_a', 'team_a_difficulty']].values
    diff_list = np.concatenate((team_h, team_a))
    diff_df = pd.DataFrame(diff_list, columns=['team', 'difficulty'])
    diff_df.sort_values(by=['difficulty'],inplace=True)
    diff_df.reset_index(inplace=True)
    diff_df.drop(columns=['index'], inplace=True)
    return diff_df

In [18]:
fixtures_df.loc[((fixtures_df['team_a'] == '1') |
                (fixtures_df['team_h'] == '1')) &
                (fixtures_df['finished'] == False),
                ['team_h','team_h_difficulty','team_a','team_a_difficulty']].head(3).replace(teams_dict)

,team_h,team_h_difficulty,team_a,team_a_difficulty
49,EVE,4,ARS,2
56,ARS,3,TOT,4
62,BOU,4,ARS,2


In [19]:
def get_gameweek(event=1):
    url = f'https://fantasy.premierleague.com/api/event/{event}/live/'
    r = requests.get(url)
    raw = json.loads(r.content)
    id_list = []
    stat_list = []
    for player in raw['elements']:
        id_list.append(player['id'])
        stat_list.append(player['stats'])
    player_stats = pd.DataFrame(stat_list)
    player_stats['id'] = id_list
    player_stats['id'] = player_stats['id'].astype(str)
    return player_stats

In [20]:
finished_events= events.loc[events['finished']]['id'].tolist()

In [21]:
element_past_fixtures = pd.DataFrame(player_summary['history'])

In [22]:
element_fixtures = pd.DataFrame(player_summary['fixtures'])

In [23]:
sample = stats_df.pop('id')

In [24]:
forwards = players.loc[players['element_type'] == 4]

In [28]:
players['now_cost'] * 0.1

0      4.4
1      4.0
2      4.4
3      5.4
4      4.8
      ... 
698    5.5
699    4.4
700    4.5
701    5.0
702    4.5
Name: now_cost, Length: 703, dtype: float64

In [31]:
element_past_fixtures

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,355,1,6,13,False,2023-08-11T19:00:00Z,0,3,1,79,2,0,1,0,0,0,0,0,0,0,3,56,67.6,23.7,48.0,13.9,1,0.60,0.05,0.65,0.23,140,0,7200159,0,0
1,355,16,15,2,True,2023-08-19T19:00:00Z,1,0,2,90,0,0,1,0,0,0,0,0,0,0,0,1,3.2,11.8,48.0,6.3,1,0.43,0.02,0.45,0.29,140,132944,8231668,161773,28829
2,355,30,17,4,False,2023-08-27T13:00:00Z,1,2,3,90,1,0,0,1,0,0,1,0,0,0,0,14,34.2,12.9,97.0,14.4,1,1.93,0.01,1.94,0.73,140,64397,8571562,91189,26792
3,355,39,10,20,True,2023-09-02T14:00:00Z,5,1,4,90,3,1,0,1,0,0,0,0,0,0,3,90,119.4,23.4,55.0,19.8,1,1.58,0.10,1.68,1.45,140,27163,8787213,53175,26012
